In [1]:
!pip install nltk
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 36.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 341.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 108.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
import json
import pandas as pd

# Function to load JSON data from a file and return it as a dictionary
def load_json_as_dict(file_path):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
    return data

# Example usage
file_path = 'VQA train.json'  # Replace with your JSON file path
train_dict = load_json_as_dict(file_path)
train_df = pd.DataFrame(train_dict)
traintrain_df.head()

In [ ]:
all_answers = {}

for answers in train_df["answers"]:
    for answer in answers:
            word = answer["answer"]
            if word in all_answers:
                all_answers[word] += 1
            else:
                all_answers[word] = 1
                
# Convert the dictionary to a DataFrame
answers_df = pd.DataFrame(list(all_answers.items()), columns=['Answer', 'Frequency'])

answers_df = answers_df.sort_values(by='Frequency', ascending=False)
# Display the first 100 rows of the DataFrame
pd.set_option('display.max_rows',100)

answers_df.head(100)

In [4]:
import torch

def check_cuda_availability():
    if torch.cuda.is_available():
        print("CUDA is available.")
        print(f"Number of CUDA devices: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"Device {i}: {torch.cuda.get_device_name(i)}")
            print(f"Memory Allocated: {torch.cuda.memory_allocated(i) / 1024 ** 2} MB")
            print(f"Memory Cached: {torch.cuda.memory_reserved(i) / 1024 ** 2} MB")
    else:
        print("CUDA is not available.")

if __name__ == "__main__":
    check_cuda_availability()

CUDA is available.
Number of CUDA devices: 1
Device 0: Tesla T4
Memory Allocated: 0.0 MB
Memory Cached: 0.0 MB


In [15]:
import re
import random
import time
from statistics import mode
import os
import json

from PIL import Image
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torch.profiler

from transformers import DistilBertTokenizer, DistilBertModel

from torch.utils.data import DataLoader





def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def process_text(text):
    # lowercase
    text = text.lower()

    # 数詞を数字に変換
    num_word_to_digit = {
        'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
        'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9',
        'ten': '10'
    }
    for word, digit in num_word_to_digit.items():
        text = text.replace(word, digit)

    # 小数点のピリオドを削除
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    # 冠詞の削除
    text = re.sub(r'\b(a|an|the)\b', '', text)

    # 短縮形のカンマの追加
    contractions = {
        "dont": "don't", "isnt": "isn't", "arent": "aren't", "wont": "won't",
        "cant": "can't", "wouldnt": "wouldn't", "couldnt": "couldn't"
    }
    for contraction, correct in contractions.items():
        text = text.replace(contraction, correct)

    # 句読点をスペースに変換
    text = re.sub(r"[^\w\s':]", ' ', text)

    # 句読点をスペースに変換
    text = re.sub(r'\s+,', ',', text)

    # 連続するスペースを1つに変換
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def get_wordnet_synset(word):
    synsets = wn.synsets(word)
    if synsets:
        return synsets[0]  # 最初のシノセットを返す
    return None

def are_semantically_similar(word1, word2):
    synset1 = get_wordnet_synset(word1)
    synset2 = get_wordnet_synset(word2)
    if synset1 and synset2:
        similarity = synset1.path_similarity(synset2)
        return similarity > 0.5 if similarity else False
    return word1 == word2  # シノセットが見つからない場合は文字列比較にフォールバック

def map_answer_to_class(answer, class_mapping):
    answer = answer.lower()
    for class_answer, class_id in class_mapping.items():
        if are_semantically_similar(answer, class_answer):
            return class_id
    return class_mapping.get("unanswerable", len(class_mapping))  # 一致するものがない場合は「unanswerable」クラスを返す



# 1. データローダーの作成
class VQADataset_preprocessed(torch.utils.data.Dataset):
    def __init__(self, processed_data_path, preprocessed_dir, answer=True):
        self.df = pd.read_json(processed_data_path)
        self.preprocessed_dir = preprocessed_dir
        self.answer = answer
        self.answer_to_class_id = {}
        self.class_id_to_answer = {}
       
        if self.answer:
            self._create_answer_mapping()
            
    def _create_answer_mapping(self):
        for answers in self.df["answers"]:
            for answer in answers:
                word = process_text(answer["answer"])
                if word not in self.answer_to_class_id:
                    class_id = len(self.answer_to_class_id)
                    self.answer_to_class_id[word] = class_id
                    self.class_id_to_answer[class_id] = word

        # Add 'unanswerable' class if not present
        if "unanswerable" not in self.answer_to_class_id:
            unknown_class_id = len(self.answer_to_class_id)
            self.answer_to_class_id["unanswerable"] = unknown_class_id
            self.class_id_to_answer[unknown_class_id] = "unanswerable"

    def update_answer_mapping(self, other_dataset):
        for word, class_id in other_dataset.answer_to_class_id.items():
            if word not in self.answer_to_class_id:
                self.answer_to_class_id[word] = class_id
                self.class_id_to_answer[class_id] = word

    def __getitem__(self, idx):
        image = torch.load(os.path.join(self.preprocessed_dir, self.df['processed_image_path'][idx]))
        question_embedding = torch.load(os.path.join(self.preprocessed_dir, self.df['processed_question_path'][idx]))

        if self.answer:
            answers = [self.answer_to_class_id[process_text(answer["answer"])] for answer in self.df["answers"][idx]]
            mode_answer_idx = mode(answers)
            return image, question_embedding, torch.Tensor(answers), int(mode_answer_idx)
        else:
            return image, question_embedding
    
    def __len__(self):
        return len(self.df)

class VQADataset(torch.utils.data.Dataset):
    def __init__(self, df_path, image_dir, tokenizer, model,transform=None, answer=True):
        self.transform = transform  # 画像の前処理
        self.image_dir = image_dir  # 画像ファイルのディレクトリ
        self.df = pd.read_json(df_path)  # 画像ファイルのパス，question, answerを持つDataFrame
        self.answer = answer

        self.tokenizer = tokenizer
        self.model = model

        self.answer_to_class_id = {}
        self.class_id_to_answer = {}
       
        # Initialize answer dictionary if answer is True
    
        if self.answer:
            self._create_answer_mapping()

    def _create_answer_mapping(self):
        for answers in self.df["answers"]:
            for answer in answers:
                word = process_text(answer["answer"])
                if word not in self.answer_to_class_id:
                    class_id = len(self.answer_to_class_id)
                    self.answer_to_class_id[word] = class_id
                    self.class_id_to_answer[class_id] = word

        # Add 'unanswerable' class if not present
        if "unanswerable" not in self.answer_to_class_id:
            unknown_class_id = len(self.answer_to_class_id)
            self.answer_to_class_id["unanswerable"] = unknown_class_id
            self.class_id_to_answer[unknown_class_id] = "unanswerable"

    def update_answer_mapping(self, other_dataset):
        for word, class_id in other_dataset.answer_to_class_id.items():
            if word not in self.answer_to_class_id:
                self.answer_to_class_id[word] = class_id
                self.class_id_to_answer[class_id] = word


    def __getitem__(self, idx):
    
        image = Image.open(f"{self.image_dir}/{self.df['image'][idx]}")
        image = self.transform(image)

        question = process_text(self.df["question"][idx])
    
        inputs = self.tokenizer(question, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs)
        question_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()

        if self.answer:
            answers = [self.answer_to_class_id[process_text(answer["answer"])] for answer in self.df["answers"][idx]]
            mode_answer_idx = mode(answers)  # 最頻値を取得（正解ラベル）

            return image, question_embedding, torch.Tensor(answers), int(mode_answer_idx)

        else:
            return image, question_embedding

    def __len__(self):
        return len(self.df)




# 3. モデルのの実装
# ResNetを利用できるようにしておく
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels: int, out_channels: int, stride: int = 1):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        out += self.shortcut(residual)
        out = self.relu(out)

        return out


class BottleneckBlock(nn.Module):
    expansion = 4

    def __init__(self, in_channels: int, out_channels: int, stride: int = 1):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))

        out += self.shortcut(residual)
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, layers):
        super().__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, layers[0], 64)
        self.layer2 = self._make_layer(block, layers[1], 128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], 256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], 512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, 512)

    def _make_layer(self, block, blocks, out_channels, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet50():
    return ResNet(BottleneckBlock, [3, 4, 6, 3])

class VQAModel(nn.Module):
    def __init__(self, n_answer: int):
        super().__init__()
        self.resnet = ResNet18()


        self.fc = nn.Sequential(
            nn.Linear(1280, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, n_answer)
        )

    def forward(self, image, question_embedding):
        image_feature = self.resnet(image)  # 画像の特徴量

        x = torch.cat([image_feature, question_embedding], dim=1)
        x = self.fc(x)

        return x
    
# 2. 評価指標の実装
# 簡単にするならBCEを利用する
def VQA_criterion(batch_pred: torch.Tensor, batch_answers: torch.Tensor):
    total_acc = 0.

    for pred, answers in zip(batch_pred, batch_answers):
        acc = 0.
        for i in range(len(answers)):
            num_match = 0
            for j in range(len(answers)):
                if i == j:
                    continue
                if pred == answers[j]:
                    num_match += 1
            acc += min(num_match / 3, 1)
        total_acc += acc / 10

    return total_acc / len(batch_pred)

# 4. 学習の実装
# Training and Evaluation Functions
def train(model, dataloader, optimizer, criterion, device):

    model.train()

    total_loss = 0
    total_acc = 0
    simple_acc = 0
    start = time.time()

    for image, question_embedding, answers, mode_answer in dataloader:
        image, question_embedding, answers, mode_answer = \
            image.to(device), question_embedding.to(device), answers.to(device), mode_answer.to(device)

        pred = model(image, question_embedding)
        loss = criterion(pred, mode_answer.squeeze())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc += VQA_criterion(pred.argmax(1), answers)
        simple_acc += (pred.argmax(1) == mode_answer).float().mean().item()
    avg_loss = total_loss / len(dataloader)
    return avg_loss, total_acc / len(dataloader), simple_acc / len(dataloader), time.time() - start
    #return total_loss / len(dataloader), total_acc / len(dataloader), simple_acc / len(dataloader), time.time() - start


def eval(model, dataloader, optimizer, criterion, device):

    model.eval()

    total_loss = 0
    total_acc = 0
    simple_acc = 0
    start = time.time()

    for image, question_embedding, answers, mode_answer in dataloader:
        image, question_embedding, answers, mode_answer = \
            image.to(device), question_embedding.to(device), answers.to(device), mode_answer.to(device)

        pred = model(image, question_embedding)
        loss = criterion(pred, mode_answer.squeeze())

        total_loss += loss.item()
        total_acc += VQA_criterion(pred.argmax(1), answers)
        simple_acc += (pred.argmax(1) == mode_answer).mean().item()
    return total_loss / len(dataloader), total_acc / len(dataloader), simple_acc / len(dataloader), time.time() - start

class EarlyStopping:
    def __init__(self, patience=3, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = 10**8
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        if self.best_loss == 10**8:
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
        elif val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print(f"Early stopping triggered. Best Loss: {self.best_loss}")
                    
                    
import torch
from torch.profiler import profile, record_function, ProfilerActivity

def train_with_profiler(model, dataloader, optimizer, criterion, device, num_epochs=1):
    model.train()
    
    # プロファイラの設定を最新のものに更新
    with torch.profiler.profile(
        activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=4, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/profiler'),
        record_shapes=True,
        profile_memory=True,
        with_stack=True
    ) as profiler:
        
        for epoch in range(num_epochs):
            for batch_idx, (image, question_embedding, answers, mode_answer) in enumerate(dataloader):
                image, question_embedding, answers, mode_answer = \
                    image.to(device), question_embedding.to(device), answers.to(device), mode_answer.to(device)
                
                with record_function("model_forward"):
                    pred = model(image, question_embedding)
                    loss = criterion(pred, mode_answer.squeeze())
                
                with record_function("backward"):
                    optimizer.zero_grad()
                    loss.backward()
                
                with record_function("optimizer_step"):
                    optimizer.step()
                
                profiler.step()
                
                #if batch_idx >= 100:  # プロファイリングを早めに終了
                #    break

    # プロファイラの結果を表示
    print(profiler.key_averages().table(sort_by="cpu_time_total", row_limit=10))
    print(profiler.key_averages().table(sort_by="cuda_time_total", row_limit=10))

    



In [9]:
import os

# ファイルのパスを指定
file_path = '/workspace/preprocessed_test/processed_data.json'

# ファイルの存在を確認
if os.path.exists(file_path):
    print("ファイルが存在します。")
else:
    print("ファイルが見つかりません。パスを再確認してください。")

# ディレクトリの内容をリスト表示
directory_path = '/path/to/your/directory'
print("ディレクトリの内容:", os.listdir(directory_path))


ファイルが存在します。


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/directory'

In [10]:
import os
print(os.getcwd())


/workspace/assets


In [16]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
import json
import numpy as np

def main():
    # デバイスの設定
    set_seed(42)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 前処理済みデータのパス
    preprocessed_train_path = "/workspace/preprocessed_train/processed_data.json"
    preprocessed_test_path = "/workspace/preprocessed_test/processed_data.json"

    # データセットの準備
    train_dataset = VQADataset_preprocessed(processed_data_path=preprocessed_train_path, preprocessed_dir="/workspace/preprocessed_train")
    test_dataset = VQADataset_preprocessed(processed_data_path=preprocessed_test_path, preprocessed_dir="/workspace/preprocessed_test", answer=False)
    
    # テストデータセットの回答マッピングをトレインデータセットから引き継ぐ
    test_dataset.update_answer_mapping(train_dataset)

    # データローダーの作成
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

    # モデルの初期化
    vqa_model = VQAModel(n_answer=len(train_dataset.answer_to_class_id)).to(device)

    # オプティマイザと損失関数
    num_epoch = 50
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(vqa_model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

    # EarlyStoppingの初期化
    early_stopping = EarlyStopping(patience=3, verbose=True)

    # モデルの保存先ディレクトリ
    model_dir = 'saved_models'
    os.makedirs(model_dir, exist_ok=True)

    # 訓練履歴を保存するリスト
    train_history = []
    
    #TRAIN profiler
    train_with_profiler(vqa_model, train_loader, optimizer, criterion, device, num_epochs=1)
    
    # トレーニング開始
    for epoch in range(num_epoch):
        train_loss, train_acc, train_simple_acc, train_time = train(vqa_model, train_loader, optimizer, criterion, device)
        
        scheduler.step(train_loss)

        # 訓練結果を保存
        train_history.append({
            'epoch': epoch + 1,
            'loss': train_loss,
            'acc': train_acc,
            'simple_acc': train_simple_acc,
            'time': train_time
        })
        
        print(f"【{epoch + 1}/{num_epoch}】\n"
              f"train time: {train_time:.2f} [s]\n"
              f"train loss: {train_loss:.4f}\n"
              f"train acc: {train_acc:.4f}\n"
              f"train simple acc: {train_simple_acc:.4f}\n"
              f"Current learning rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Early stopping と最良モデルの保存
        if train_loss < early_stopping.best_loss:
            early_stopping.best_loss = train_loss
            early_stopping.counter = 0
            torch.save(vqa_model.state_dict(), os.path.join(model_dir, "best_model.pth"))
            print(f"New best model saved with loss: {early_stopping.best_loss:.4f}")
        else:
            early_stopping.counter += 1
            if early_stopping.counter >= early_stopping.patience:
                print(f"Early stopping triggered. No improvement for {early_stopping.patience} epochs.")
                break

    # 訓練履歴を保存
    with open('train_history.json', 'w') as f:
        json.dump(train_history, f)

    # 最良モデルを読み込む
    print(f"Loading best model with loss: {early_stopping.best_loss:.4f}")
    vqa_model.load_state_dict(torch.load(os.path.join(model_dir, "best_model.pth")))

    # 提出用ファイルの作成
    vqa_model.eval()
    submission = []
    with torch.no_grad():
        for image, question in test_loader:
            image, question = image.to(device), question.to(device)
            pred = vqa_model(image, question)
            pred_class_ids = pred.argmax(1).cpu().tolist()
            submission.extend([train_dataset.class_id_to_answer[class_id] for class_id in pred_class_ids])

    submission = np.array(submission)
    np.save("submission.npy", submission)
    print("Submission file created with best model.")

if __name__ == "__main__":
    main()

STAGE:2024-07-15 07:14:49 133:133 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-07-15 07:14:53 133:133 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-07-15 07:14:53 133:133 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         0.06%       2.327ms        79.21%        2.964s     740.948ms       0.000us         0.00%        1.011s     252.739ms           0 b           0 b      -1.50 Mb    -591.05 M

In [4]:

def main():
    # デバイスの設定
    set_seed(42)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # データ変換
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # データパス
    df_path_train = "VQA train.json"
    image_dir_train = "train"
    df_path_test = "VQA valid.json"
    image_dir_test = "valid"
    
    # トークナイザーとモデルのロード
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertModel.from_pretrained("distilbert-base-uncased")

    # データセットの準備
    train_dataset = VQADataset(df_path=df_path_train, image_dir=image_dir_train, transform=transform, tokenizer=tokenizer, model=model)
    test_dataset = VQADataset(df_path=df_path_test, image_dir=image_dir_test, transform=transform, tokenizer=tokenizer, model=model, answer=False)
    
    # テストデータセットの回答マッピングをトレインデータセットから引き継ぐ
    test_dataset.update_answer_mapping(train_dataset)

    # データローダーの作成
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=2, prefetch_factor=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2, prefetch_factor=2)

    # モデルの初期化
    vqa_model = VQAModel(n_answer=len(train_dataset.answer_to_class_id)).to(device)

    # オプティマイザと損失関数
    num_epoch = 50
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(vqa_model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

    # EarlyStoppingの初期化
    early_stopping = EarlyStopping(patience=5, verbose=True)

    # モデルの保存先ディレクトリ
    model_dir = 'saved_models'
    os.makedirs(model_dir, exist_ok=True)

    # 訓練履歴を保存するリスト
    train_history = []

    # トレーニング開始
    with torch.profiler.profile(
        activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log'),
        record_shapes=True,
        profile_memory=True,
        with_stack=True
    ) as prof:
        for epoch in range(num_epoch):
            train_loss, train_acc, train_simple_acc, train_time = train(vqa_model, train_loader, optimizer, criterion, device)
            
            scheduler.step(train_loss)

            # 訓練結果を保存
            train_history.append({
                'epoch': epoch + 1,
                'loss': train_loss,
                'acc': train_acc,
                'simple_acc': train_simple_acc,
                'time': train_time
            })
            
            print(f"【{epoch + 1}/{num_epoch}】\n"
                  f"train time: {train_time:.2f} [s]\n"
                  f"train loss: {train_loss:.4f}\n"
                  f"train acc: {train_acc:.4f}\n"
                  f"train simple acc: {train_simple_acc:.4f}\n"
                  f"Current learning rate: {optimizer.param_groups[0]['lr']:.6f}")
            
            # Early stopping と最良モデルの保存
            if train_loss < early_stopping.best_loss:
                early_stopping.best_loss = train_loss
                early_stopping.counter = 0
                torch.save(vqa_model.state_dict(), os.path.join(model_dir, "best_model.pth"))
                print(f"New best model saved with loss: {early_stopping.best_loss:.4f}")
            else:
                early_stopping.counter += 1
                if early_stopping.counter >= early_stopping.patience:
                    print(f"Early stopping triggered. No improvement for {early_stopping.patience} epochs.")
                    break
            
            prof.step()
            

    # 訓練履歴を保存
    with open('train_history.json', 'w') as f:
        json.dump(train_history, f)

    # 最良モデルを読み込む
    print(f"Loading best model with loss: {early_stopping.best_loss:.4f}")
    vqa_model.load_state_dict(torch.load(os.path.join(model_dir, "best_model.pth")))

    # 提出用ファイルの作成
    vqa_model.eval()
    submission = []
    with torch.no_grad():
        for image, question in test_loader:
            image, question = image.to(device), question.to(device)
            pred = vqa_model(image, question)
            pred_class_id = pred.argmax(1).cpu().item()
            submission.append(train_dataset.class_id_to_answer[pred_class_id])

    submission = np.array(submission)
    np.save("submission.npy", submission)
    print("Submission file created with best model.")

if __name__ == "__main__":
    main()


TypeError: VQADataset.__init__() got an unexpected keyword argument 'df_path'